# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logRegCode').getOrCreate()

In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
data = spark.read.csv('customer_churn.csv',header=True, inferSchema=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [6]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [9]:
data.groupBy('Company')

In [10]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'],outputCol='features')

In [11]:
data_assembled = assembler.transform(data)

In [13]:
data_assembled.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- features: vector (nullable = true)



In [14]:
final_data = data_assembled.select('features','Churn')

In [15]:
train_data , test_data = final_data.randomSplit([.7,.3])

In [27]:
log_reg= LogisticRegression(labelCol='Churn')

In [28]:
log_model = log_reg.fit(train_data)

In [29]:
test_predict = log_model.evaluate(test_data)

In [34]:
test_predict.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,9090.43,1.0...|    0|[1.47882604477129...|[0.81439519621939...|       0.0|
|[28.0,11128.95,1....|    0|[4.25029751053582...|[0.98594049759433...|       0.0|
|[28.0,11204.23,0....|    0|[1.86160532259703...|[0.86548395176080...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.66762148963169...|[0.99069284875313...|       0.0|
|[29.0,10203.18,1....|    0|[3.84565884722567...|[0.97907490197330...|       0.0|
|[29.0,12711.15,0....|    0|[5.66773192163832...|[0.99655620651743...|       0.0|
|[29.0,13255.05,1....|    0|[4.21895174627159...|[0.98549930363122...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.64478152906189...|[0.97453812806704...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.02012416836468...|[0.99757651959989...|       0.0|
|[30.0,10183.98,

In [30]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [35]:
binary_eval = BinaryClassificationEvaluator(labelCol='Churn',rawPredictionCol='prediction')

In [36]:
binary_eval.evaluate(test_predict.predictions)

0.7770091256991462

In [37]:
newCustomerdata = spark.read.csv('new_customers.csv',header=True, inferSchema=True)

In [38]:
final_model = log_reg.fit(final_data)

In [39]:
newdata_assembled = assembler.transform(newCustomerdata)

In [44]:
newCustomer_data = newdata_assembled.select('features')

In [47]:
finalPredict = final_model.transform(newdata_assembled)

In [50]:
finalPredict.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

